In [5]:
%pip install Beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [7]:
!playwright install

In [13]:
# Import libraries
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import Constants

In [ ]:
# Get the html
async def get_html(url, selector, sleep = 5, retries = 3):
    html = None
    
    for i in range(1, retries + 1):
        
        # Create a sleep period, so we don't get banned from sites for scraping
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                
                # Print scraped data
                print(await page.title())
                html = await page.inner_html(selector)
                
        except PlaywrightTimeout:
            # If tomeout, show me
            print(f"Timeout error on {url}")
            continue
        else:
            break
        
    return html
                

In [24]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]

    standing_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standing_pages:
        
        # save scraped data in directory
        save_path = os.path.join(Constants.STANDINGS_DIR, url.split("/")[-1])
        
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [25]:
for season in Constants.SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Refere

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Refere

In [26]:
standings_files = os.listdir(Constants.STANDINGS_DIR)

In [35]:
standings_file = os.path.join(Constants.STANDINGS_DIR, standings_files[0])
with open(standings_file, 'r') as f:
    html = f.read()
    
soup = BeautifulSoup(html)
links = soup.find_all("a")
hrefs = [l.get("href") for l in links]
box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

In [36]:
box_scores

['https://www.basketball-reference.com/boxscores/202110190MIL.html',
 'https://www.basketball-reference.com/boxscores/202110190LAL.html',
 'https://www.basketball-reference.com/boxscores/202110200CHO.html',
 'https://www.basketball-reference.com/boxscores/202110200DET.html',
 'https://www.basketball-reference.com/boxscores/202110200NYK.html',
 'https://www.basketball-reference.com/boxscores/202110200TOR.html',
 'https://www.basketball-reference.com/boxscores/202110200MEM.html',
 'https://www.basketball-reference.com/boxscores/202110200MIN.html',
 'https://www.basketball-reference.com/boxscores/202110200NOP.html',
 'https://www.basketball-reference.com/boxscores/202110200SAS.html',
 'https://www.basketball-reference.com/boxscores/202110200UTA.html',
 'https://www.basketball-reference.com/boxscores/202110200POR.html',
 'https://www.basketball-reference.com/boxscores/202110200PHO.html',
 'https://www.basketball-reference.com/boxscores/202110210ATL.html',
 'https://www.basketball-reference

In [ ]:
for url in box_scores:
    save_path = os.path.join(Constants.SCORES_DIR, url.split("/")[-1])
    
    if os.path.exists(save_path):
        continue
        
    html = await get_html(url, "#content")
    if not html:
        continue
    with opne(save_path, "w+") as f:
        f.write(html)